In [1]:
import platform, os, sys, datetime, re
import multiprocessing
from os.path import join
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
# sys.path.append(join(root_dir,'cvtracer'))
from cvt.TrAQ.Trial import Trial
from cvt.TrAQ.Tank import Tank
from cvt.TrAQ.Group import Group
from cvt.TrAQ.CVTracer import CVTracer, create_named_window, wait_on_named_window
from cvt.utils import *


default_settings = dict(
    t_start        = 0,     # Time at which to start tracking, in seconds.
    t_end          = -1,    # Time at which to end tracking, in seconds.
    
    # Background subtraction (for naive background subtraction only).
    bkg_frame_skip = 100,   # Using every frame of the video to compute the background takes a while.
                            # Instead we only use one frame in bkg_frame_skip.
    bkg_sub_amp    = 4,     # Contrast amplification factor applied after background subtraction.
    
    # Contour detection.
    n_pixel_blur   =  7,    # square-root of n-pixels for threshold blurring
    block_size     = 15,    # contour block size
    thresh_offset  = 15,    # threshold offset for contour-finding
    min_area       = 25,    # minimum area for threhold detection
    max_area       = 60,    # maximum area for threhold detection
    RGB            = False, # track in color, false does greyscale
    online_viewer  = False, # Toggle live preview of tracking.

    # What information to draw on the tracking output video.
    video_output_options = dict(tank=True, repeat_contours=False, all_contours=True, 
                                contour_color=(0,200,255), contour_thickness=1, 
                                points=False, directors=True, timestamp=True)
    )

settings_list = list(default_settings.keys()) + \
                ['input_file', 'tracking_dir', 'output_dir', 'trial_file',  
                 'new_input_file', 'bkg_file', 'tank_file', 'settings_file', 
                 'tank_radius', 'ext', 'filename', 'pop', 'age', 'group', 'Nfish', 
                 'Nframes', 'fps', 'fourcc', 'width', 'height']

def create_settings_(input_file, tracking_dir, settings):
    
    globals().update(settings)
    tank_diameter_vs_age = { 7:9.6, 14:10.4, 21:12.8, 28:17.7, 42:33.8 }
    
    ''' Extract trial info from the filename and the video itself. '''

    filename,ext = os.path.splitext(os.path.basename(input_file))
    pop,_,age,group,Nfish,_ = filename.split('_')
    Nfish    = int(re.findall('\d+',Nfish)[0])
    age      = int(age[:-3])
    tank_radius = tank_diameter_vs_age[age]/2

    cap      = cv2.VideoCapture(input_file)
    cap.read()
    Nframes  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps      = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc   = int(cap.get(cv2.CAP_PROP_FOURCC))
    width    = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    
    ''' Define and create necessary folders/files/links. '''
    
    output_dir = join(tracking_dir,filename)

    new_input_file = input_file
    if not 'windows' in platform.system().lower():
        new_input_file = join(output_dir,'raw'+ext)
    
    settings_file = join(output_dir,'tracking_settings.txt')
    trial_file    = join(output_dir,'trial.pik')
    tank_file     = join(output_dir,'tank.pik')
    bkg_file      = join(output_dir,f'background-{bkg_frame_skip}.npy')
    
    for k,v in locals().items():
        if k not in ['cap','settings','tank_diameter_vs_age','_']:
            settings[k] = v
        
    return settings


def create_directories(settings):
    for k in 'tracking_dir','output_dir','input_file','new_input_file':
        globals()[k] = settings[k]
    if not os.path.exists(tracking_dir):
        os.mkdir(tracking_dir)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    if not 'windows' in platform.system().lower():
        if not os.path.exists(new_input_file):
            os.symlink(os.path.relpath(input_file,output_dir),new_input_file)
    return


def save_settings(settings):
    with open(settings['settings_file'],'w') as fh:
        for k,v in settings.items():
            print(f'{k} = {v}',file=fh)
    return

# Locate input/output and allocate CPU's

`tracking_dir` sets the top-level output directory. The output of tracking each video will go in `tracking_dir`, in a subdirectory named after the input video file.  

`n_threads` controls the number of tracking tasks to execute simulataneously. `n_threads = None` defaults to the number of CPU's on the machine running the notebook.

`input_files` set the list of video files to perform tracking on.

In [2]:
tracking_dir = '../tracking/partial'

def create_settings(input_file,tracking_dir=tracking_dir,settings=default_settings):
    return create_settings_(input_file,tracking_dir,settings)

n_threads = None

# input_files = sorted(glob('../raw_videos/*.avi'))
# input_files = sorted(glob('../raw_videos/*_*_7dpf_*.avi'))
input_files = []
for f in sorted(glob('../raw_videos/*.avi')):
    print(f)
    settings = create_settings(f)
    if not os.path.exists(settings['trial_file']):
        input_files.append(f)

input_files = input_files[:1]
display(input_files)

# input_files = ['../raw_videos/SF_Sat_14dpf_GroupA_n1_2020-06-13-110945-0000.avi']
# input_files = ['../raw_videos/SF_Sat_14dpf_GroupA_n2a_2020-06-13-090438-0000.avi']
# input_files = ['../raw_videos/SF_Sat_14dpf_GroupB_n2a_2020-06-13-115534-0000.avi']
# input_files = ['../raw_videos/broken/Pa_Fri_14dpf_groupA_n5_15FPS.avi']

../raw_videos/Pa_Fri_14dpf_GroupA_n2_15FPS.avi
../raw_videos/Pa_Fri_14dpf_GroupA_n2b_15FPS.avi
../raw_videos/Pa_Fri_14dpf_GroupA_n2b_30FPS-0000.avi
../raw_videos/Pa_Fri_14dpf_GroupB_n2_-0000-0000.avi
../raw_videos/Pa_Fri_14dpf_GroupB_n2b_0000.avi
../raw_videos/Pa_Fri_14dpf_GroupC_n2_-0000-0000.avi
../raw_videos/Pa_Fri_14dpf_GroupD_n2_-0000-0000.avi
../raw_videos/Pa_Fri_14dpf_GroupD_n2b_-0000.avi
../raw_videos/Pa_Fri_7dpf_GroupA_n2_2020-06-05-120920-0000.avi
../raw_videos/Pa_Fri_7dpf_GroupA_n2b_2020-06-05-103456-0000.avi
../raw_videos/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000.avi
../raw_videos/Pa_Fri_7dpf_GroupB_n2b_2020-06-05-114635-0000.avi
../raw_videos/Pa_Fri_7dpf_GroupB_n5_2020-06-05-094643-0000.avi
../raw_videos/SF_Sat_14dpf_GroupA_n1_2020-06-13-110945-0000.avi
../raw_videos/SF_Sat_14dpf_GroupA_n2a_2020-06-13-090438-0000.avi
../raw_videos/SF_Sat_14dpf_GroupA_n2b_2020-06-13-110452-0000.avi
../raw_videos/SF_Sat_14dpf_GroupA_n5_2020-06-13-120445-0000.avi
../raw_videos/SF_Sat_14dpf

['../raw_videos/Pa_Fri_14dpf_GroupA_n2_15FPS.avi']

# Locate the tanks

In [5]:
for input_file in input_files:
    print(input_file)
    settings = create_settings(input_file)
    create_directories(settings)
    globals().update(settings)
    tank = Tank(r_cm=tank_radius)
    tank.load_or_locate_and_save(tank_file,input_file)

# Compute backgrounds

Compute the background by averaging frames over the entire video. Save it as `background.npy` in the output directory. Use the pre-existing file if there is one.

In [6]:
def subtract_background(frame,bkg,bkg_sub_amp):
    return 255-np.minimum(255,bkg_sub_amp*np.absolute(frame-bkg)).astype(np.uint8)    


def compute_background(settings):
    
    globals().update(settings)
    
    if os.path.exists(bkg_file):
        bkg  = np.load(bkg_file)
    else:
        t0    = datetime.datetime.now()
        cap   = cv2.VideoCapture(new_input_file)
        _,frame = cap.read()
        bkg   = np.zeros(frame.shape,dtype=float)
        count = 0
        # If bkg_frame_skip is small (<10) it may be faster to use
        # cap.grab instead of cap.set.
        for n in range(0,Nframes,bkg_frame_skip):
            cap.set(cv2.CAP_PROP_POS_FRAMES,n)
            ret,frame = cap.read()
            bkg      += frame
            count    += 1
        bkg   = bkg / count
        np.save(bkg_file,bkg)
        print(input_file)
        print('Using every {bkg_frame_skip}th frame.')
        print(datetime.datetime.now()-t0)
        sys.stdout.flush()

    # Show the background.
    plt.figure(figsize=(9,9))
    plt.imshow(bkg.astype(np.uint))
    plt.savefig(join(output_dir,'background.png'),dpi=150)
#     plt.show()
    plt.clf()

#---------------------------------------------------

# # Single thread version.
# for f in input_files:
#     settings = create_settings(f)
#     compute_background(settings)

#----

# Multithread version.
with multiprocessing.Pool(n_threads) as pool:
    pool.map(compute_background,[create_settings(f) for f in input_files])

### Track with simple background subtraction

In [7]:
def track(settings):
    
    save_settings(settings)
    globals().update(settings)
    
    trial = Trial()
    trial.init(video_file=new_input_file, output_dir=output_dir, n=Nfish, 
               fps=fps, tank_radius=tank_radius, t_start=t_start, t_end=t_end)

    cvt = CVTracer(trial, n_pixel_blur=n_pixel_blur, block_size=block_size, 
                   threshold_offset=thresh_offset, min_area=min_area, RGB=True,
                   online=online_viewer)
    
    bkg = np.load(bkg_file)
    
    try:
        cvt.set_frame(cvt.frame_start)
        for i_frame in range(cvt.frame_start, cvt.frame_end+1):
            if cvt.get_frame():
                cvt.frame = subtract_background(cvt.frame,bkg,bkg_sub_amp)
                cvt.mask_tank()
                cvt.detect_contours()
                cvt.analyze_contours()
                cvt.connect_frames()
                cvt.update_trial()
                cvt.draw(**video_output_options)
                cvt.write_frame()
                if not cvt.post_frame(delay=1):
                    break
                cvt.print_current_frame()
        cvt.release()
        cvt.trial.save()
    except:
        %tb
        cvt.release()
    
    return

#---------------------------------------------------

tweaks = dict(t_start = 60, t_end = 70)

#---------------------------------------------------

# # Single thread version.
# for f in input_files:
#     print(input_file)
#     settings = create_settings(f)
#     track(settings)

#----

# Multithread version.
def task(input_file):
    print(input_file)
    sys.stdout.flush()
    settings = create_settings(input_file)
    settings.update(tweaks)
    track(settings)

with multiprocessing.Pool(n_threads) as pool:
    pool.map(task,input_files)

In [ ]:
# cv2.destroyAllWindows()